# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

Note: Make sure that you have PostgreSQL running.

## Imports

In [635]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Generate Test Data

In [ ]:
# Create two blobs of data
prev_month_blobs = h.generate_random_polygons(n=10000)
curr_month_blobs = h.generate_random_polygons(n=10000)

In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
6557,"POLYGON ((-97.86914 26.73168, -97.86914 26.731...",858982f0-34d8-4fc6-b2a3-e49ea4a2db93,9uf2hfmu
7146,"POLYGON ((-97.87858 26.95112, -97.87858 26.951...",14d08727-b223-4fdd-a354-b5d8a6e01bdc,9uf3kdr4
7931,"POLYGON ((-98.17879 26.99046, -98.17879 26.990...",5e34d5f2-a790-4232-9c8d-08644768a7ea,9uf1tckx
7610,"POLYGON ((-98.33691 27.27728, -98.33691 27.277...",4b9120db-fa0d-4363-bb9f-c46d95918fc4,9uf54m9r
4987,"POLYGON ((-97.80132 27.48900, -97.80132 27.489...",db23a2fd-c61e-4836-97ff-edce895767d1,9ufkqkpr


In [ ]:
curr_month_blobs.head()

,geometry,id,geohash
2408,"POLYGON ((-97.85179 26.59206, -97.85179 26.592...",f5415bb4-e38b-4210-8055-e3cafb0c0479,9udrm2fz
7133,"POLYGON ((-97.90133 26.78679, -97.90133 26.787...",604d6d8e-14f6-48d0-ad30-ae57f649f392,9uf2khqw
2680,"POLYGON ((-98.05069 26.88622, -98.05069 26.887...",95fe4c40-09fd-4d8c-9e3f-e6f16929602b,9uf2by9f
5879,"POLYGON ((-98.26851 27.26602, -98.26851 27.266...",76a1ffe4-92e5-4626-9729-2655ade15b91,9uf55geq
2612,"POLYGON ((-98.05534 27.37249, -98.05534 27.372...",d70a5485-1929-4f4b-a2a4-2fcaff1e1f39,9uf78xn8


In [ ]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [ ]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
6557,"POLYGON ((-97.86914 26.73168, -97.86914 26.731...",858982f0-34d8-4fc6-b2a3-e49ea4a2db93,9uf2hfmu
7146,"POLYGON ((-97.87858 26.95112, -97.87858 26.951...",14d08727-b223-4fdd-a354-b5d8a6e01bdc,9uf3kdr4
7931,"POLYGON ((-98.17879 26.99046, -98.17879 26.990...",5e34d5f2-a790-4232-9c8d-08644768a7ea,9uf1tckx
7610,"POLYGON ((-98.33691 27.27728, -98.33691 27.277...",4b9120db-fa0d-4363-bb9f-c46d95918fc4,9uf54m9r
4987,"POLYGON ((-97.80132 27.48900, -97.80132 27.489...",db23a2fd-c61e-4836-97ff-edce895767d1,9ufkqkpr


## Non-optimized process

### 1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [ ]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [ ]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

Function `convert_col_to_string` executed in 0.0490 sec, CPU: 26.90%, Memory: 0.86MB
Function `convert_col_to_string` executed in 0.0516 sec, CPU: 53.50%, Memory: 14.22MB
                                               geometry  \
6557  POLYGON ((-97.869141 26.731683, -97.869141 26....   
7146  POLYGON ((-97.878577 26.951121, -97.878577 26....   
7931  POLYGON ((-98.178786 26.990459, -98.178786 26....   
7610  POLYGON ((-98.336905 27.27728, -98.336905 27.2...   
4987  POLYGON ((-97.801315 27.489001, -97.801315 27....   

                                        id   geohash  
6557  858982f0-34d8-4fc6-b2a3-e49ea4a2db93  9uf2hfmu  
7146  14d08727-b223-4fdd-a354-b5d8a6e01bdc  9uf3kdr4  
7931  5e34d5f2-a790-4232-9c8d-08644768a7ea  9uf1tckx  
7610  4b9120db-fa0d-4363-bb9f-c46d95918fc4  9uf54m9r  
4987  db23a2fd-c61e-4836-97ff-edce895767d1  9ufkqkpr  


/Users/sra/files/projects/matching_optimization/helpers.py:305: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))
/Users/sra/files/projects/matching_optimization/helpers.py:305: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))


### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [ ]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0026 sec, CPU: 0.00%, Memory: 0.30MB
Function `df_itertuple` executed in 0.0024 sec, CPU: 0.00%, Memory: 0.06MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [ ]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.0345 sec, CPU: 0.00%, Memory: 0.00MB


Create and insert into tables:

In [ ]:
prev_month_blobs.head(1)

,geometry,id,geohash
6557,"POLYGON ((-97.86914 26.73168, -97.86914 26.731...",858982f0-34d8-4fc6-b2a3-e49ea4a2db93,9uf2hfmu


In [ ]:
prev_month_blobs_wkt[0]

('POLYGON ((-97.869141 26.731683, -97.869141 26.731994, -97.868777 26.731994, -97.868777 26.731683, -97.869141 26.731683))',
 '858982f0-34d8-4fc6-b2a3-e49ea4a2db93',
 '9uf2hfmu')

In [ ]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.
Inserted 10000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 0.4518 sec, CPU: 0.00%, Memory: 0.00MB
Table curr_blobs_wkt truncated.
Inserted 10000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 0.3738 sec, CPU: 36.10%, Memory: 13.91MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [ ]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 10000 records from prev_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0454 sec, CPU: 5.90%, Memory: 0.52MB
Retrieved 10000 records from curr_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0430 sec, CPU: 33.30%, Memory: 0.78MB


Compare the tables before and after for a sanity check:

In [ ]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-97.869141 26.731683, -97.869141 26....",858982f0-34d8-4fc6-b2a3-e49ea4a2db93,9uf2hfmu
1,"POLYGON ((-97.878577 26.951121, -97.878577 26....",14d08727-b223-4fdd-a354-b5d8a6e01bdc,9uf3kdr4
2,"POLYGON ((-98.178786 26.990459, -98.178786 26....",5e34d5f2-a790-4232-9c8d-08644768a7ea,9uf1tckx
3,"POLYGON ((-98.336905 27.27728, -98.336905 27.2...",4b9120db-fa0d-4363-bb9f-c46d95918fc4,9uf54m9r
4,"POLYGON ((-97.801315 27.489001, -97.801315 27....",db23a2fd-c61e-4836-97ff-edce895767d1,9ufkqkpr


In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
6557,"POLYGON ((-97.86914 26.73168, -97.86914 26.731...",858982f0-34d8-4fc6-b2a3-e49ea4a2db93,9uf2hfmu
7146,"POLYGON ((-97.87858 26.95112, -97.87858 26.951...",14d08727-b223-4fdd-a354-b5d8a6e01bdc,9uf3kdr4
7931,"POLYGON ((-98.17879 26.99046, -98.17879 26.990...",5e34d5f2-a790-4232-9c8d-08644768a7ea,9uf1tckx
7610,"POLYGON ((-98.33691 27.27728, -98.33691 27.277...",4b9120db-fa0d-4363-bb9f-c46d95918fc4,9uf54m9r
4987,"POLYGON ((-97.80132 27.48900, -97.80132 27.489...",db23a2fd-c61e-4836-97ff-edce895767d1,9ufkqkpr


In [ ]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 10000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


It worked!

### 2. Matching

We will match the polygons using GeoPandas.

In [ ]:
postgresql_details = h.pg_details()
h.run_parallel_matching(table_prev='prev_blobs_wkt', 
                        table_curr='curr_blobs_wkt', 
                        output_table='matched_results', 
                        postgresql_details=postgresql_details, 
                        db_name='blob_matching', 
                        num_workers=4, 
                        batch_size=100)

Table matched_results created successfully.
Retrieved 10000 records from prev_blobs_wkt.
Function `run_parallel_matching` executed in 42.9475 sec, CPU: 40.50%, Memory: 514.14MB


geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.